In [15]:
%pip -q install mlflow boto3 awscli

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.2.3 requires docutils<0.22,>=0.20, but you have docutils 0.19 which is incompatible.


In [13]:
import mlflow

mlflow.set_tracking_uri("http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/")

with mlflow.start_run():
    mlflow.log_param("param1", 15)
    mlflow.log_metric("metric1", 0.89)

🏃 View run bustling-wren-346 at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/0/runs/06e9df695d67482383180f13a18ae777
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/0


## Import Dependencies

In [14]:
# Data manipulation and analysis
import numpy as np
import pandas as pd
import re
from collections import Counter

# Machine Learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix,
    precision_recall_fscore_support
)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Natural Language Processing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# MLflow for experiment tracking
import mlflow
import mlflow.sklearn

# AWS Services (optional - for S3 artifact storage)
import boto3

# Download required NLTK data
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

# Set visualization style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 10

print("✓ All libraries imported successfully")

✓ All libraries imported successfully


## Mlflow Configuration

In [16]:
MLFLOW_TRACKING_URI = "http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

print(f"✓ MLflow tracking URI set to: {MLFLOW_TRACKING_URI}")

# Create or set experiment
EXPERIMENT_NAME = "RF Baseline"
mlflow.set_experiment(EXPERIMENT_NAME)

print(f"✓ MLflow experiment set to: {EXPERIMENT_NAME}")

# Test MLflow connection
print("\n--- Testing MLflow Connection ---")
try:
    with mlflow.start_run(run_name="Connection_Test") as run:
        mlflow.log_param("test_param", "connection_successful")
        mlflow.log_metric("test_metric", 1.0)
        run_id = run.info.run_id
    print(f"✓ MLflow connection successful! Run ID: {run_id}")
    print(f"  View at: {MLFLOW_TRACKING_URI}#/experiments/0/runs/{run_id}")
except Exception as e:
    print(f"✗ MLflow connection failed: {str(e)}")

✓ MLflow tracking URI set to: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/


2025/11/11 16:51:33 INFO mlflow.tracking.fluent: Experiment with name 'RF Baseline' does not exist. Creating a new experiment.


✓ MLflow experiment set to: RF Baseline

--- Testing MLflow Connection ---
🏃 View run Connection_Test at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/407855132886144546/runs/5b6a09e2046a4d4ca88db854652fdaad
🧪 View experiment at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/407855132886144546
✓ MLflow connection successful! Run ID: 5b6a09e2046a4d4ca88db854652fdaad
  View at: http://ec2-54-211-18-166.compute-1.amazonaws.com:5000/#/experiments/0/runs/5b6a09e2046a4d4ca88db854652fdaad


## Data Loading

In [17]:
# Load the dataset
DATA_URL = 'https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv'
df = pd.read_csv(DATA_URL)

print(f"✓ Dataset loaded successfully")
print(f"  Shape: {df.shape[0]} rows × {df.shape[1]} columns")
print(f"\n  First 5 rows:")
print(df.head())

print(f"\n  Columns: {list(df.columns)}")
print(f"  Target distribution:")
print(df['category'].value_counts().sort_index())


✓ Dataset loaded successfully
  Shape: 37249 rows × 2 columns

  First 5 rows:
                                       clean_comment  category
0   family mormon have never tried explain them t...         1
1  buddhism has very much lot compatible with chr...         1
2  seriously don say thing first all they won get...        -1
3  what you have learned yours and only yours wha...         0
4  for your own benefit you may want read living ...         1

  Columns: ['clean_comment', 'category']
  Target distribution:
category
-1     8277
 0    13142
 1    15830
Name: count, dtype: int64
